In [ ]:
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, History, Callback
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
x_train = np.load("/content/drive/MyDrive/Tesis Yolla/Output/200 files 1/200_train_image.npy")
r_train = np.load("/content/drive/MyDrive/Tesis Yolla/Output/200 files 1/200_train_region.npy")
b_train = np.load("/content/drive/MyDrive/Tesis Yolla/Output/200 files 1/200_train_boundary.npy")

In [ ]:
x_train = np.expand_dims(x_train, axis=3).astype('float32')
r_train = np.expand_dims(r_train, axis=3).astype('float32')
b_train = np.expand_dims(b_train, axis=3).astype('float32')

In [ ]:
def residual_block(x, filter_amount, filter_size):
    residual = BatchNormalization()(x)
    residual = Activation(activation = 'relu')(residual)
    residual = Conv2D(filters = filter_amount, kernel_size = filter_size, padding = 'same', strides = (1,1))(residual)
    residual = Dropout(0.2)(residual)
    residual = BatchNormalization()(residual)
    residual = Activation(activation = 'relu')(residual)
    residual = Conv2D(filters = filter_amount, kernel_size = filter_size, padding = 'same', strides = (1,1))(residual)
    residual = Dropout(0.2)(residual)
    
    res_shortcut = Conv2D(filters = filter_amount, kernel_size = 1, padding = 'same', strides = (1,1))(x)
    res_shortcut = BatchNormalization()(res_shortcut)
    
    residual = add([res_shortcut, residual])
    return residual

In [ ]:
def residual_block2(x, y, filter_amount, filter_size):
    residual = Concatenate()([x, y])
    residual = BatchNormalization()(residual)
    residual = Activation(activation = 'relu')(residual)
    residual = Conv2D(filters = filter_amount, kernel_size = filter_size, padding = 'same', strides = (1,1))(residual)
    residual = Dropout(0.2)(residual)
    residual = BatchNormalization()(residual)
    residual = Activation(activation = 'relu')(residual)
    residual = Conv2D(filters = filter_amount, kernel_size = filter_size, padding = 'same', strides = (1,1))(residual)
    residual = Dropout(0.2)(residual)
    
    res_shortcut = Conv2D(filters = filter_amount, kernel_size = 1, padding = 'same', strides = (1,1))(x)
    res_shortcut = BatchNormalization()(res_shortcut)
    
    residual = add([res_shortcut, residual])
    return residual

In [ ]:
def rfcn(inputshape, filter_amount, filter_size):
    #Encoder
    inputs = Input(shape = inputshape)
    res1 = residual_block(inputs, filter_amount[0], filter_size)
    down_conv1 = Conv2D(filters = filter_amount[1], kernel_size = 2, padding = 'valid', strides = (2,2))(res1)
    res2 = residual_block(down_conv1, filter_amount[1], filter_size)
    down_conv2 = Conv2D(filters = filter_amount[2], kernel_size = 2, padding = 'valid', strides = (2,2))(res2)
    res3 = residual_block(down_conv2, filter_amount[2], filter_size)
    down_conv3 = Conv2D(filters = filter_amount[3], kernel_size = 2, padding = 'valid', strides = (2,2))(res3)
    
    #Bridge
    res4 = residual_block(down_conv3, filter_amount[3], filter_size)
    up_conv1 = Conv2DTranspose(filters = filter_amount[2], kernel_size = 2, padding = 'valid', strides = (2,2))(res4)
    
    #Decoder
    res5 = residual_block2(up_conv1, res3, filter_amount[2], filter_size)
    up_conv2 = Conv2DTranspose(filters = filter_amount[1], kernel_size = 2, padding = 'valid', strides = (2,2))(res5)
    res6 = residual_block2(up_conv2, res2, filter_amount[1], filter_size)
    up_conv3 = Conv2DTranspose(filters = filter_amount[0], kernel_size = 2, padding = 'valid', strides = (2,2))(res6)
    res6 = residual_block2(up_conv3, res1, filter_amount[0], filter_size)
    
    #Region
    conv1 = Conv2D(filters = filter_amount[0], kernel_size = 3, padding = 'same', strides = (1,1))(res6)
    do = Dropout(0.2)(conv1)
    output_region = Conv2D(filters = 1, kernel_size = 1, activation='sigmoid', kernel_initializer='he_normal', name = 'region_output')(do)
    
    #Boundary
    conv2 = Conv2D(filters = filter_amount[0], kernel_size = 3, padding = 'same', strides = (1,1))(res6)
    do = Dropout(0.2)(conv2)
    output_boundary = Conv2D(filters = 1, kernel_size = 1, activation='sigmoid', kernel_initializer='he_normal', name = 'boundary_output')(do)
    
    return Model(inputs = [inputs], outputs=[output_region, output_boundary])
    

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())


In [ ]:
def binary_focal_loss(gamma, alpha, beta):
    """
    Binary form of focal loss.
         Focal loss for binary classification problems
    
    focal_loss(p_t) = -alpha_t * (1 - p_t)**gamma * log(p_t)
        where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    alpha = tf.constant(alpha, dtype=tf.float32)
    gamma = tf.constant(gamma, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        """
        y_true shape need be (None,1)
        y_pred need be compute after sigmoid
        """
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
    
        p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(K.ones_like(y_true)-y_pred) + K.epsilon()
        fl = - alpha_t * K.pow((K.ones_like(y_true)-p_t),gamma) * K.log(p_t)
        fl = K.mean(fl)
        fl = beta * fl
        return fl
    return binary_focal_loss_fixed

In [ ]:
filter_amount = [32, 64, 128, 256]
filter_size = 3
model = rfcn(inputshape = (192, 256, 1), filter_amount = filter_amount, filter_size = filter_size)
model_checkpoint = ModelCheckpoint('200 RFNC + DAL (Region 0.6, Boundary 0.4).h5', monitor='loss', save_best_only=True)
model.compile(optimizer = Adam(lr = 0.001), 
              loss = {'region_output' : binary_focal_loss(gamma = 2., alpha = 0.25, beta = 0.6), 'boundary_output' : binary_focal_loss(gamma = 2., alpha = 0.25, beta = 0.4)}, 
              metrics = {'region_output' : dice_coef, 'boundary_output' : dice_coef})
history_model = model.fit([x_train], [r_train, b_train], batch_size=48, epochs=1000, verbose=1, shuffle = True, callbacks=[model_checkpoint])



In [ ]:
print(history_model.history.keys())

In [ ]:
epoch_count = range(1, len(history_model.history['loss']) + 1)
plt.plot(epoch_count, history_model.history['loss'])
plt.title('Training Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
epoch_count = range(1, len(history_model.history['region_output_loss']) + 1)
plt.plot(epoch_count, history_model.history['region_output_loss'])
plt.title('Region Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
epoch_count = range(1, len(history_model.history['boundary_output_loss']) + 1)
plt.plot(epoch_count, history_model.history['boundary_output_loss'])
plt.title('Boundary Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(epoch_count, history_model.history['region_output_dice_coef'])
plt.plot(epoch_count, history_model.history['boundary_output_dice_coef'])
plt.legend(['Region Dice Score', 'Boundary Dice Score'])
plt.title('Training Dice Score')
plt.ylabel('Dice SCore')
plt.xlabel('epoch')
plt.show()

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
x_test = np.load("/content/drive/MyDrive/Tesis Yolla/Output/200 files 1/200_test_image.npy")
r_test = np.load("/content/drive/MyDrive/Tesis Yolla/Output/200 files 1/200_test_region.npy")
b_test = np.load("/content/drive/MyDrive/Tesis Yolla/Output/200 files 1/200_test_boundary.npy")

In [ ]:
x_test = np.expand_dims(x_test, axis=3).astype('float32')
r_test = np.expand_dims(r_test, axis=3).astype('float32')
b_test = np.expand_dims(b_test, axis=3).astype('float32')

In [ ]:
result = model.predict(x_test)

In [ ]:
gabung = []
for i in range(len(r_test)):
  g = result[0][i] + result[1][i]
  # g = asarray(g)
  g[g < 1] = 0
  g[g >= 1] = 1
  gabung.append(g)

In [ ]:
gabung2 = []
for i in range(len(r_test)):
  g = result[0][i] + result[1][i]
  # g = asarray(g)
  # g[g < 1] = 0
  # g[g >= 1] = 1
  gabung2.append(g)

In [ ]:
dice_r = dice_coef(result[0], r_test)
precision_r = precision_m(result[0], r_test)
recall_r = recall_m(result[0], r_test)

dice_b = dice_coef(result[1], b_test)
precision_b = precision_m(result[1], b_test)
recall_b = recall_m(result[1], b_test)

dice_g = dice_coef(gabung, r_test)
precision_g = precision_m(gabung, r_test)
recall_g = recall_m(gabung, r_test)

dice_g2 = dice_coef(gabung2, r_test)
precision_g2 = precision_m(gabung2, r_test)
recall_g2 = recall_m(gabung2, r_test)

In [ ]:
with tf.compat.v1.Session() as sess:
  print('----Region----\nDice Coef Score\t: ' +str(dice_r.numpy())+ '\nPrecision\t: ' +str(precision_r.numpy())+ '\nRecall\t\t: '+str(recall_r.numpy()))
  print('\n----Boundary----\nDice Coef Score\t: ' +str(dice_b.numpy())+ '\nPrecision\t: ' +str(precision_b.numpy())+ '\nRecall\t\t: '+str(recall_b.numpy()))
  print('\n----Gabungan----\nDice Coef Score\t: ' +str(dice_g.numpy())+ '\nPrecision\t: ' +str(precision_g.numpy())+ '\nRecall\t\t: '+str(recall_g.numpy()))
  print('\n----Gabungan 2----\nDice Coef Score\t: ' +str(dice_g2.numpy())+ '\nPrecision\t: ' +str(precision_g2.numpy())+ '\nRecall\t\t: '+str(recall_g2.numpy()))

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(len(result[0])):
  plt.subplot(8,5,i+1),plt.imshow(result[0][i].reshape(192,256),cmap = 'gray')
  plt.title(i+1), plt.xticks([]), plt.yticks([])
plt.show()